# Εισαγωγή των απαραίτητων βιβλιοθηκών
- Pandas: Χρησιμοποιείται για την επεξεργασία και ανάλυση δεδομένων.
- Requests: Χρησιμοποιείται για την αποστολή HTTP αιτημάτων και την αλληλεπίδραση με διαδικτυακές διεπαφές προγραμματισμού εφαρμογών (APIs).
- Re: Παρέχει υποστήριξη για τις κανονικές εκφράσεις στην Python.

In [ ]:
import pandas as pd
import requests, re

# Φόρτωση δεδομένων
Το απόσπασμα κώδικα διαβάζει τα δεδομένα από δύο αρχεία CSV, το `ratings.csv` και το `books.csv`, και δημιουργεί pandas DataFrames.

1. `pd.read_csv('training/ratings.csv')`: Αυτή η γραμμή διαβάζει τα δεδομένα από το αρχείο `ratings.csv` που βρίσκεται στη διαδρομή `'.training/ratings.csv'` χρησιμοποιώντας τη συνάρτηση `pd.read_csv()` από τη βιβλιοθήκη pandas. Τα δεδομένα φορτώνονται σε ένα DataFrame με όνομα `ratings`.

2. `pd.read_csv('training/books.csv')`: Αντίστοιχα, αυτή η γραμμή διαβάζει τα δεδομένα από το αρχείο `books.csv` που βρίσκεται στη διαδρομή `'training/books.csv'` και τα αποθηκεύει σε ένα DataFrame με όνομα `books`.

Με την εκτέλεση αυτού του κώδικα, το DataFrame `ratings` γεμίζει με τα δεδομένα από το αρχείο `ratings.csv`, το DataFrame `books` γεμίζει με τα δεδομένα από το αρχείο `books.csv`

In [ ]:
ratings = pd.read_csv('training/ratings.csv')
books = pd.read_csv('training/books.csv')

Εκτύπωση Σύνοψης Δεδομένων

In [1]:
print(ratings.head())
print(ratings.describe())

NameError: name 'ratings' is not defined

In [ ]:
print(books.head())
print(books.describe())

# Ανάλυση Αξιολογήσεων Χρηστών και Ορίζοντες Αριθμού Αξιολογήσεων

- Ανάλυση Αξιολογήσεων Χρηστών

Αυτό το τμήμα κώδικα αναλύει τις αξιολογήσεις των χρηστών μετρώντας τον αριθμό των αξιολογήσεων ανά χρήστη και υπολογίζοντας ορίζοντες αριθμού αξιολογήσεων. Έπειτα, μετρά τον αριθμό των χρηστών με λιγότερες από X αξιολογήσεις και υπολογίζει το ποσοστό του συνόλου των χρηστών για κάθε όριο. Τέλος, δημιουργεί και εκτυπώνει ένα DataFrame που περιλαμβάνει τα αποτελέσματα.

In [ ]:
user_ratings = ratings.groupby('user_id')['rating'].count()
user_rating_counts = ratings['user_id'].value_counts()
users_with_ratings = user_rating_counts.groupby(user_ratings).count()
rating_thresholds = list(range(5, 100, 5))

count_per_threshold = []
previous_count = 0
total_users = 53424  
for threshold in rating_thresholds:
    count = user_ratings[user_ratings < threshold].count() - previous_count
    count_per_threshold.append(count)
    previous_count += count

percent_per_threshold = [round((count / total_users) * 100) for count in count_per_threshold]

df = pd.DataFrame({"fewer than X": rating_thresholds, "count": count_per_threshold, "percent": percent_per_threshold})

(df)


- Φιλτράρισμα Χρηστών με βάση τον Αριθμό Αξιολογήσεων

Αυτό το τμήμα κώδικα φιλτράρει τους χρήστες με λιγότερες από έναν καθορισμένο αριθμό αξιολογήσεων. Δημιουργεί ένα νέο DataFrame με την ονομασία filtered_ratings αποκλείοντας τους χρήστες των οποίων ο αριθμός αξιολογήσεων είναι μικρότερος από το όριο φίλτρου. Στη συνέχεια, αναθέτει νέα αναγνωριστικά χρήστη βασισμένα στον ομαδοποιημένο δείκτη των αναγνωριστικών χρηστών.

Στη συνέχεια, μετρά τον αριθμό των αξιολογήσεων ανά χρήστη στο DataFrame filtered_ratings χρησιμοποιώντας τις συναρτήσεις groupby() και size(). Το αποτέλεσμα αποθηκεύεται σε ένα νέο DataFrame με την ονομασία rating_counts.

Τέλος, δημιουργεί ένα DataFrame με την ονομασία users αναθέτοντας σε αυτό το DataFrame το rating_counts. Προσθέτει επίσης μια νέα στήλη με την ονομασία 'new_data' που έχει την τιμή False. Στη συνέχεια, εμφανίζονται οι πρώτες γραμμές του DataFrame users.

In [ ]:
filter_out= 15
filtered_ratings = ratings[~ratings['user_id'].isin(user_rating_counts[user_rating_counts < filter_out].index.tolist())]
filtered_ratings.loc[:, 'user_id'] = filtered_ratings.groupby('user_id').ngroup()


rating_counts = filtered_ratings.groupby('user_id').size().reset_index(name='rating_count')

users = pd.DataFrame(rating_counts)
ratings = filtered_ratings
users['new_data'] = False
users.head()

In [ ]:
books=pd.read_csv('books.csv')

# Αναζήτηση για Εικόνες Εξωφύλλων Βιβλίων και Συνδέσμους Amazon

Αυτό το τμήμα κώδικα ανακτά εικόνες εξωφύλλων βιβλίων και συνδέσμους Amazon για τα βιβλία στο DataFrame των βιβλίων.

1. Επαναλαμβάνεται για κάθε γραμμή στο DataFrame και ελέγχει αν ο σύνδεσμος amazon_link ξεκινά με 'https://www.amazon.com/'.

2. Αν όχι, πραγματοποιεί αναζήτηση Google για την εικόνα εξωφύλλου του βιβλίου χρησιμοποιώντας τον τίτλο του βιβλίου.

3. Ο τίτλος του βιβλίου επεξεργάζεται αφαιρώντας ειδικούς χαρακτήρες και κενά, μετατρέπεται σε πεζά γράμματα και αντικαθίσταται τα κενά με παύλες.

4. Δημιουργείται ο όρος αναζήτησης και αποστέλλεται ένα αίτημα GET στο Google Custom Search API.

5. Η απόκριση αναλύεται ως JSON και ανακτώνται οι λίστες στοιχείων αποτελεσμάτων αναζήτησης.

6. Ελέγχει κάθε στοιχείο για ένα στοιχείο που έχει εκχύλισμα με σύνδεσμο εικόνας και, αν βρεθεί, τον αναθέτει στο image_url.

7. Επίσης, ελέγχει αν υπάρχει ένας σύνδεσμος Amazon στα στοιχεία και, αν βρεθεί, τον αναθέτει στο amazon_link και σταματά τη διαδικασία αναζήτησης.

8. Τέλος, ενημερώνονται οι στήλες image_url και amazon_link στο DataFrame των βιβλίων.

Αυτός ο κώδικας ενισχύει το DataFrame των βιβλίων προσθέτοντας εικόνες εξωφύλλων βιβλίων και τους αντίστοιχους συνδέσμους Amazon για τα βιβλία που αρχικά δεν είχαν έγκυρους συνδέσμους Amazon.

In [ ]:
#API_key
with open('../api_key', 'rb') as key_file:
    api_key = key_file.read().decode()
with open('../search_engine_id', 'rb') as key_file:
    search_engine_id = key_file.read().decode()

for index, row in books.iterrows():
    amazon_link = str(row['amazon_link'])

    if not amazon_link.startswith('https://www.amazon.com/'):
        book_title = row['title']
        # Remove special characters and spaces
        search_title = re.sub(r'[^\w\s-]', '', book_title)

        # Replace spaces with hyphens
        search_title = re.sub(r'\s', '+', search_title)

        # Convert to lowercase
        
        search_title = search_title.lower()
        search_term = f"{search_title}+book+cover+amazon"
        # Construct the search URL
        search_url = f"https://www.googleapis.com/customsearch/v1?key={api_key}&cx={search_engine_id}&q={search_term}"
        # Perform the search and retrieve the image URLs
        response = requests.get(search_url)
        search_results = response.json()
        items = search_results.get("items", [])  # Get the list of items from the search results

        image_url = None

        for item in items:
            pagemap = item.get("pagemap", {})  # Get the pagemap dictionary of the item
            scraped = pagemap.get("scraped", [])  # Get the list of scraped items

            if scraped:
                image_link = scraped[0].get("image_link")  # Get the image link from the scraped item
                if image_link:
                    image_url = image_link  # Found an image link, assign it to image_url

                link = item.get("link")  # Get the link from the item
                if link and link.startswith('https://www.amazon.com/'):
                    amazon_link = link  # Found an Amazon link, assign it to amazon_link
                    break    
        books.at[index, 'image_url'] = image_url
        books.at[index, 'amazon_link'] = amazon_link


In [ ]:
ratings.to_csv('ratings.csv',index=False)
books.to_csv('books.csv',index=False)
users.to_csv('users.csv',index=False)
